In [9]:
from itertools import combinations
from pyspark import SparkContext
from collections import defaultdict
import sys
import csv
import time

In [12]:
'''
filter_threshold = int(sys.argv[1])
threshold = sys.argv[2]
input_file_path = sys.argv[3]
output_file_path = sys.argv[4]
'''
filter_threshold = 20
threshold = 50
input_file_path = "/Users/gopi/Desktop/Assignment2/input2.csv"
output_file_path = "/Users/gopi/Desktop/Assignment2/output2.csv"
sc = SparkContext.getOrCreate()
sc.setLogLevel("ERROR")

In [13]:
def apriori(partition, total_length):
    baskets = list(partition)
    #print(baskets)
    unique_items = set()
    for basket in baskets:
        for item in basket:
            unique_items.add(item)
    single_item_sets = list(unique_items)#list(set(i for sub in baskets for i in sub))
    scaled_threshold = (len(baskets) / total_length) * threshold
    #scaled_threshold = int(threshold / num_partitions)
    #print(scaled_threshold)
    candidates = list(combinations(single_item_sets,1))
    support = get_support_candidates(candidates, baskets)
    pruned_candidates = prune_candidates(support, scaled_threshold)
    #print(pruned_candidates)
    
    result_set = []
    result_set.append(pruned_candidates)
    count = 2
    while(len(pruned_candidates) > 0) :
        time1 = time.time()
        item_sets = combinations(pruned_candidates, 2)
        item_sets = reformat_item_sets(item_sets, count)
        #print(item_sets)
        support = get_support_candidates(item_sets, baskets)
        pruned_candidates = prune_candidates(support, scaled_threshold)
        if len(pruned_candidates) > 0:
            result_set.append(pruned_candidates)
        print("time for 1 iteration is:" , (time.time() - time1))
        count+=1
    #print(result_set)
    return result_set   
    
def reformat_item_sets(item_sets, size):
    result_dict = {}
    for item in item_sets:
        new_tuple = tuple(set(item[0]).union(set(item[1])))
        if(size == len(new_tuple)):
            new_tuple = tuple(sorted(new_tuple))
            if new_tuple not in result_dict:
                result_dict[new_tuple] = None
    return list(result_dict.keys())

def get_single_item_sets(baskets) :
    result = set()
    for basket in baskets:
        for item in basket:
            result.add(item)
    return result

def get_support_candidates(candidates, baskets) :
    support = {}
    for candidate in candidates:
        candidate_set = set(candidate)
        #print(candidate_set)
        for basket in baskets:
            if candidate_set.issubset(set(basket)):
                support[candidate] = support[candidate] + 1 if candidate in support else 1
    #print(support)
    return support

def prune_candidates(items, support):
    filtered_keys = list(dict(filter(lambda item : item[1] >= support, items.items())).keys())
    filtered_keys = sorted(filtered_keys)
    return filtered_keys

def son(partition, candidates):
    result = {}
    baskets = list(partition)
    result = get_support_candidates(candidates, baskets)
    return list(result.items())

def write_data_to_file(file_path, file_mode, item_sets, data_type):
    file=open(file_path,file_mode);
    file.write(data_type);
    file.write("\n")
    
    result = defaultdict(lambda : [])
    
    for item in item_sets:
        length = len(item)
        if(length == 1):
            result[length].append("(" + item[0] + ")")
        else:
            result[length].append(str(item))
    keys = sorted(list(result.keys()))
    
    for key in keys:
        string = ",".join(sorted(result[key]))
        file.write(str(string))
        file.write("\n\n")
    
    file.close()
    
def custom_hash(x) :
    return hash(x[0])

In [14]:
start_time = time.time()

'''
initial_rdd = sc.textFile(input_file_path)
first = initial_rdd.first()
intermediate_data = initial_rdd.filter(lambda x : x != first).map(lambda line: line.split(",")).map(lambda x : [x[0].replace('\"', ''), x[1].replace('\"', '').lstrip('0'), x[5].replace('\"', '').lstrip('0')]).map(lambda x : [x[0] + "-" + x[1],  x[2]]).collect()
'''

intermediate_data = []
with open(input_file_path,'r') as input_file:
    reader = csv.reader(input_file)
    headers = next(reader)
    for row in reader:
        date = row[0].replace('\"', '')
        customer_id = row[1].replace('\"', '').lstrip('0')
        product_id = row[5].replace('\"', '').lstrip('0')
        intermediate_data.append([date + "-" + customer_id, product_id])

header = ['DATE-CUSTOMER_ID','PRODUCT_ID']

with open("customer_product_csv.csv", 'w', newline = '') as file:
    write = csv.writer(file)
    write.writerow(header)
    write.writerows(intermediate_data)
    file.close()
    
print("time taken for pre_processing is : ", (time.time() - start_time))

rdd = sc.textFile('customer_product.csv')
first = rdd.first()
rdd = rdd.filter(lambda x : x != first).map(lambda row : row.split(",")).map(lambda row : (row[0], row[1])).groupByKey().mapValues(set).filter(lambda row : len(row[1]) > filter_threshold).map(lambda row : (row[0], row[1]))
num_partitions = rdd.getNumPartitions()
total_length = rdd.count()
if num_partitions < int(total_length / 1000):
    rdd = rdd.partitionBy(int(total_length / 1000))
rdd = rdd.map(lambda x : x[1]).cache()
time1 = time.time()
num_partitions = rdd.getNumPartitions()
print(total_length)
candidate_items = rdd.mapPartitions(lambda partition : apriori(partition, total_length)).flatMap(lambda result : result).distinct().collect()
print(rdd.take(5))
write_data_to_file(output_file_path, "w", candidate_items, "Candidates:")
print("time taken for candidate items is :", time.time() -time1)
time2 = time.time()
#print(candidate_items)

son = rdd.mapPartitions(lambda partition : son(partition, candidate_items)).reduceByKey(lambda a,b : a+b).filter(lambda frequent_item : frequent_item[1] >= threshold).map(lambda frequent_item : frequent_item[0]).collect()
write_data_to_file(output_file_path, "a", son, "Frequent Itemsets:")
print("Duration:" ,time.time() - start_time)
#print(son)



time taken for pre_processing is :  1.7194669246673584


/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


5652


time for 1 iteration is: 140.58526992797852                         (0 + 5) / 5]
time for 1 iteration is: 0.056791067123413086
time for 1 iteration is: 0.004128217697143555
time for 1 iteration is: 2.1457672119140625e-06
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
time for 1 iteration is: 141.78237390518188                         (1 + 4) / 5]
time for 1 iteration is: 0.03862118721008301
time for 1 iteration is: 0.003522157669067383
time for 1 iteration is: 145.9035358428955                          (2 + 3) / 5]
time for 1 iteration is: 0.07236599922180176
time for 1 iteration is: 0.017204999923706055
time for 1 iteration is: 0.006567239761352539
time for 1 iteration is: 0.004725933074951172
time for 1 iteration is: 154.103924036026===>                       (3 + 2) / 5]
time for 1 iteration is: 0.06432676315307617
time for 1 iteration is: 0.008909940719604492
tim

[{'4710015103479', '4710105045450', '4710160013067', '4712814168888', '4710632001622', '4710094020568', '4710199010174', '4710943101059', '4710160013074', '4710008242154', '4710883000061', '4710363522007', '4710731041802', '4710105045320', '4710085120093', '4711186319911', '4710321871055', '4713428000151', '4712277820637', '4710036006223', '4710386123229', '4710043105124', '4710883000092'}, {'16379130125', '4711080010112', '8851991511127', '4716426880411', '4710126194298', '4711109808058', '4717265991788', '4715874000235', '4710583350718', '4711713390031', '4715398208339', '4902738431155', '4712815116192', '4710094021572', '4712820617769', '20149772', '4710144101452', '4710017089085', '4713089003003', '4903301563495', '4710154012144', '4710543214012', '4715398106864', '4712815116185', '4710035310055', '4719860715110'}, {'2250271000218', '4711080010112', '4710105010175', '4710543111014', '4710030343508', '4719090105002', '4710861700020', '4710105015125', '4710088436344', '4710088410382'

/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


Duration: 169.81505298614502


In [27]:
print(num_partitions)

2
